<a target="_blank" href="https://colab.research.google.com/drive/1CAg-Kb7QvhkuIgfyjxNodvFL9jzGR7oJ?usp=share_link">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
%%capture
! pip install pip==21.3.1
! pip install ultralytics==8.0.53
! pip install clearml==1.9.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import clearml
from clearml import Task
from clearml import Model
from ultralytics import YOLO

In [ ]:
clearml.browser_login()

<IPython.core.display.Javascript object>


🤖 ClearML connected successfully - let's build something! 🚀


In [ ]:
# get best performing clearml task
task = Task.get_task(
    project_name="Pedestrian-Detection-YOLOv8", 
    task_name="yolov8s_imgsz1024_epochs50",
)
# download the best model from clearml server
best_model_id = task.output_models_id["best"]
clearml_model = Model(best_model_id)
checkpoint_fp = clearml_model.get_local_copy()
print(checkpoint_fp)

2023-03-18 20:05:28,094 - clearml.storage - INFO - Downloading: 5.00MB / 21.47MB @ 16.63MBs from https://files.clear.ml/Pedestrian-Detection-YOLOv8/yolov8s_imgsz1024_epochs50.5b25d74ef1cc4f5686b1a424b6296095/models/best.pt
2023-03-18 20:05:28,155 - clearml.storage - INFO - Downloading: 10.00MB / 21.47MB @ 81.98MBs from https://files.clear.ml/Pedestrian-Detection-YOLOv8/yolov8s_imgsz1024_epochs50.5b25d74ef1cc4f5686b1a424b6296095/models/best.pt
2023-03-18 20:05:28,259 - clearml.storage - INFO - Downloading: 15.00MB / 21.47MB @ 48.16MBs from https://files.clear.ml/Pedestrian-Detection-YOLOv8/yolov8s_imgsz1024_epochs50.5b25d74ef1cc4f5686b1a424b6296095/models/best.pt
2023-03-18 20:05:28,315 - clearml.storage - INFO - Downloading: 20.00MB / 21.47MB @ 89.68MBs from https://files.clear.ml/Pedestrian-Detection-YOLOv8/yolov8s_imgsz1024_epochs50.5b25d74ef1cc4f5686b1a424b6296095/models/best.pt
2023-03-18 20:05:28,331 - clearml.storage - INFO - Downloaded 21.47 MB successfully from https://files.cl

In [ ]:
%%shell
cp /content/drive/MyDrive/Reza/Projects/Pedestrian-Detection/data/citypersons.zip .
unzip -q citypersons.zip

In [ ]:
# load yolo model from downloaded checkpoint
model = YOLO(checkpoint_fp)

In [ ]:
img_fp = "/content/citypersons/valid/images/frankfurt_000000_002196_leftImg8bit.png"
%timeit result = model.predict(img_fp)


image 1/1 /content/citypersons/valid/images/frankfurt_000000_002196_leftImg8bit.png: 512x1024 5 pedestrians, 748.7ms
Speed: 1.8ms preprocess, 748.7ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_002196_leftImg8bit.png: 512x1024 5 pedestrians, 757.9ms
Speed: 1.5ms preprocess, 757.9ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_002196_leftImg8bit.png: 512x1024 5 pedestrians, 715.3ms
Speed: 1.5ms preprocess, 715.3ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_002196_leftImg8bit.png: 512x1024 5 pedestrians, 700.3ms
Speed: 1.5ms preprocess, 700.3ms inference, 1.3ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/citypersons/valid/images/frankfurt_000000_002196_leftImg8bit.png: 512x1024 5 pedestrians, 752.0ms
Speed: 1.7

946 ms ± 218 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
img_fp = "/content/citypersons/valid/images/frankfurt_000001_016029_leftImg8bit.png"
result = model.predict(img_fp)


image 1/1 /content/citypersons/valid/images/frankfurt_000001_016029_leftImg8bit.png: 512x1024 3 riders, 11 pedestrians, 953.2ms
Speed: 7.2ms preprocess, 953.2ms inference, 33.6ms postprocess per image at shape (1, 3, 1024, 1024)


In [ ]:
# evaluate model performance on validation set
model.val("/content/citypersons/dataset.yaml")

Ultralytics YOLOv8.0.53 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/citypersons/valid/labels.cache... 441 images, 10 backgrounds, 0 corrupt: 100%|██████████| 451/451 [00:00<?, ?it/s]
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [08:24<00:00, 17.41s/it]
                   all        451       4164      0.721      0.516      0.593      0.405
                 rider        451       1007      0.674       0.38      0.441      0.289
            pedestrian        451       3157      0.768      0.653      0.744      0.521
Speed: 9.1ms preprocess, 1076.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val4


In [ ]:
# save the checkpoint to GDrive
! cp "{checkpoint_fp}" /content/drive/MyDrive/Reza/Projects/Pedestrian-Detection/models/yolov8s_imgsz1024_mAP50_0.593.pt